In [95]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
from datetime import datetime
import os
import sys
import re
import yaml

In [96]:
# MinIO Storage Settings
MINIO_ENDPOINT = os.getenv("AWS_S3_ENDPOINT")
MINIO_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
MINIO_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
LOGS_BUCKET = "logs"
REPORTS_BUCKET = "report"
PLAYBOOKS_BUCKET = "playbook"
LOG_FILE = "service_error_down.txt" # This variable will be used throughout the script

# LLM Info
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

In [97]:
def extract_and_save_keywords(report_filename="incident_report.txt", output_filename="extracted_keywords.json"):
    """Reads the full report, extracts keywords, and saves them to a new file."""
    print(f"\n--> Step 1: Extracting and saving keywords...")
    details = {}

    with open(report_filename, 'r', encoding='utf-8') as f:
        report_text = f.read()
    # Define regex patterns for host, service, and optionally a port
    host_pattern = re.search(r"AFFECTED HOST:\s*(.*)", report_text, re.IGNORECASE)
    service_pattern = re.search(r"AFFECTED SERVICE:\s*(.*)", report_text, re.IGNORECASE)
    port_pattern = re.search(r"port (\d+)", report_text, re.IGNORECASE)

    if host_pattern:
        details["host"] = host_pattern.group(1).strip().strip('*').strip()
    if service_pattern:
        raw_service = service_pattern.group(1).strip().strip('*').strip()
        details["service"] = raw_service.replace('.service', '') if raw_service.endswith('.service') else raw_service
        
    # Set a default port and override if found in the text
    details["port"] = 80
    if port_pattern:
        details["port"] = int(port_pattern.group(1))
        
    if "host" in details and "service" in details:
        print(f"    ✅ Extracted Host: '{details['host']}', Service: '{details['service']}', Port: {details['port']}")
        with open(output_filename, 'w') as out_f:
            json.dump(details, out_f, indent=4)
        print(f"    ✅ Successfully saved keywords to '{output_filename}'.")
        return details
        
    print("    ❌ Failed to extract keywords.")
    return None

def query_model_for_playbook_tasks(incident_report="incident_report.txt"):
    """Queries the AI model to generate only the 'tasks' section of an Ansible playbook."""
    print(f"\n--> Step 2: Querying AI model for playbook tasks...")
    with open(incident_report, 'r', encoding='utf-8') as f:
        report_content = f.read()
    
    prompt = f"""
Based on the full incident report below, generate a list of Ansible tasks for remediation.
Your response MUST be ONLY the YAML for the 'tasks' section. Start your response with '- name: ...'.
Do NOT include the playbook header (name, hosts, become, vars).
Do NOT include seperator like '---'
Make sure when you write the lines to call an ansible module for example 'ansible.builtin.dnf:' , always include the colon.

**Full Incident Report for Context:**
---
{report_content}
---

**Example of a valid response format:**
- name: Ensure httpd is installed
  ansible.builtin.dnf:
    name: httpd
    state: present
- name: Start and enable httpd service
  ansible.builtin.service:
    name: httpd
    state: started
    enabled: yes
"""
    headers = {"Content-Type": "application/json"}
    payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": 1024}
    
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status()
        result = response.json()
        if 'choices' in result and result['choices']:
            tasks_yaml = result['choices'][0].get('text', '').strip()
            # Clean any extra text the model might add
            if "```" in tasks_yaml:
                 tasks_yaml = re.search(r"```(?:\w*\n)?(.*)```", tasks_yaml, re.DOTALL).group(1)
            print("    ✅ AI-generated tasks received.")
            return tasks_yaml
    except Exception as e:
        print(f"    ❌ Error generating playbook tasks from AI model: {e}")
    return None

def construct_final_playbook(tasks_yaml, keywords):
    """Constructs the final playbook by combining the static header and the AI-generated tasks."""
    print("\n--> Step 3: Constructing final playbook...")
    
    # Create the static header using the extracted keywords
    header = {
        'name': f"Restore {keywords.get('service', 'service').upper()} Service",
        'hosts': keywords.get('host'),
        'become': True,
        'vars': {
            'max_retries': 3,
            'retry_delay': 10,
            'service': keywords.get('service'),
            'port': keywords.get('port')
        }
    }

    print(tasks_)
    
    try:
        # Clean the raw YAML text from the model before parsing
        cleaned_tasks_yaml = "\n".join(line for line in tasks_yaml.splitlines() if not line.strip().startswith('**'))
        #tasks = yaml.safe_load(cleaned_tasks_yaml)
        final_playbook_yaml = [header | {'tasks': cleaned_tasks_yaml}]
        print(yaml.dump(final_playbook_yaml, sort_keys=False))
        # Combine the header and tasks into a single playbook structure
        # final_playbook_data = [{**header, 'tasks': tasks_yaml}]
        # final_playbook_yaml = yaml.dump(final_playbook_data, sort_keys=False, indent=2)
        
        print("    ✅ Final playbook constructed successfully.")
        return final_playbook_yaml
    except Exception as e:
        print(f"    ❌ Error constructing final playbook: {e}")
    return None


def upload_to_minio(s3_client, bucket, object_name, content):
    """Uploads content to a specified MinIO bucket."""
    print(f"--> Uploading '{object_name}' to bucket '{bucket}'...")
    try:
        s3_client.put_object(Body=content.encode('utf-8'), Bucket=bucket, Key=object_name)
        print(f"    ✅ Successfully uploaded '{object_name}'.")
        return True
    except Exception as e:
        print(f"    ❌ Error uploading to MinIO: {e}")
        return False


In [98]:
def main():
    
    s3_client = boto3.client(
    's3', 
    endpoint_url=MINIO_ENDPOINT, 
    aws_access_key_id=MINIO_ACCESS_KEY, 
    aws_secret_access_key=MINIO_SECRET_KEY
    )
    
    keywords = extract_and_save_keywords()
    if not keywords:
        sys.exit("Pipeline stopped: Failed to extract keywords.")
        
    tasks_yaml = query_model_for_playbook_tasks()
    print(tasks_yaml)
    if not tasks_yaml: sys.exit("Pipeline stopped: Could not generate playbook tasks.")

    final_playbook = construct_final_playbook(tasks_yaml, keywords)
    if not final_playbook: sys.exit("Pipeline stopped: Could not construct final playbook.")

        
    if not upload_to_minio(s3_client, PLAYBOOKS_BUCKET, f"remediation_playbook_{datetime.now().strftime('%Y%m%d_%H%M%S')}.yml", final_playbook):
        sys.exit("Pipeline stopped: Failed to upload playbook.")

    print("\n--- FINAL ACTIONABLE PLAYBOOK ---")
    print(final_playbook)

if __name__ == "__main__":
    print("==============================================")
    print("AIOps PIPELINE STAGE: Generate Remediation Playbook")
    print("==============================================")
    main()
    print("\n==============================================")
    print("            STAGE COMPLETE")
    print("==============================================")

AIOps PIPELINE STAGE: Generate Remediation Playbook

--> Step 1: Extracting and saving keywords...
    ✅ Extracted Host: 'aiops', Service: 'httpd', Port: 80
    ✅ Successfully saved keywords to 'extracted_keywords.json'.

--> Step 2: Querying AI model for playbook tasks...


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


    ✅ AI-generated tasks received.
[TASKS]
- name: Ensure httpd is installed
  ansible.builtin.dnf:
    name: httpd
    state: present
- name: Start and enable httpd service
  ansible.builtin.service:
    name: httpd
    state: started
    enabled: yes
- name: Check HTTP server status
  ansible.builtin.command: sudo systemctl status httpd
- name: Verify HTTP server is listening on Port 80
  ansible.builtin.command: sudo ss -tln | grep 80
[/TASKS]

--> Step 3: Constructing final playbook...


NameError: name 'tasks_' is not defined